# Import necessary lib. 


In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.contrib.layers import l1_l2_regularizer

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

import os
import h5py
import time
# from batchnorm import batch_norm,fc_batch_norm

from sklearn.metrics import accuracy_score,classification_report
import Model_1226 as my_model

E:\Liulicong\Software\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


# Configuration
Set the class names and num (17).
Set the data path (relative path). 

In [2]:
# configuration
# data attribution
CLASS_NUM = 17
CLASS_URBAN = ['Urban_Compact high-rise', 'Urban_Compact mid-rise', 'Urban_Compact low-rise',
               'Urban_Open high-rise', 'Urban_Open mid-rise', 'Urban_Open low-rise',
               'Urban_Lightweight low-rise', 'Large low-rise', 'Sparesly built', 'Heavy industry']
CLASS_RURAL = ['Dense trees', 'Scattered trees', 'Bush and scrub', 'Low plants', 'Bare rock or paved', 'Bare soil or sand', 'Water']
CLASS_NAME = CLASS_URBAN + CLASS_RURAL

# data path
base_dir = r'../data/'
path_training = os.path.join(base_dir, 'training.h5')
path_validation = os.path.join(base_dir, 'validation.h5')
path_test = os.path.join(base_dir, 'round1_test_a_20181109.h5')
# random
np.random.seed(0)

# Read Data
Check the data foramt.

In [3]:
h5py_training = h5py.File(path_training,'r')
h5py_validation = h5py.File(path_validation,'r')
h5py_test = h5py.File(path_test, 'r')

### get s1 image channel data
### it is not really loaded into memory. only the indexes have been loaded.
print("-" * 60)
print("training part")
s1_training = h5py_training['sen1']
print('s1_training: ', s1_training.shape)
s2_training = h5py_training['sen2']
print('s2_training: ', s2_training.shape)
label_training = h5py_training['label']
print('label_training: ', label_training.shape)

print("-" * 60)
print("validation part")
s1_validation = h5py_validation['sen1']
print('s1_validation: ', s1_validation.shape)
s2_validation = h5py_validation['sen2']
print('s2_validation: ', s2_validation.shape)
label_validation = h5py_validation['label']
print('label_validation: ', label_validation.shape)

print("-" * 60)
print("Test part")
s1_test = h5py_test['sen1']
print('s1_test: ', s1_test.shape)
s2_test = h5py_test['sen2']
print('s2_test: ', s2_test.shape)


train_num, pixel_x, pixel_y, sar_bands = s1_training.shape
vali_num, pixel_x, pixel_y, optical_bands = s2_validation.shape
test_num = s1_test.shape[0]

------------------------------------------------------------
training part
s1_training:  (352366, 32, 32, 8)
s2_training:  (352366, 32, 32, 10)
label_training:  (352366, 17)
------------------------------------------------------------
validation part
s1_validation:  (24119, 32, 32, 8)
s2_validation:  (24119, 32, 32, 10)
label_validation:  (24119, 17)
------------------------------------------------------------
Test part
s1_test:  (4838, 32, 32, 8)
s2_test:  (4838, 32, 32, 10)


# Pre Training

# CV Training

In [16]:
MODEL_Prefix = "SHUFFLE_FALSE_5CV"
N = 5
kflod_turns = 1

In [17]:
from sklearn.model_selection import StratifiedKFold
for kflod_turn in range(kflod_turns):
    sfolder = StratifiedKFold(n_splits=N, random_state=kflod_turn, shuffle=False)
    for i, (train_loc, vali_loc) in enumerate(sfolder.split(np.zeros((vali_num, 1)), np.zeros(vali_num))):
        _id = i + N * kflod_turn
        print(_id, '; Train: %s | test: %s' % (train_loc, vali_loc))
        X_optical_train = s2_validation[train_loc, :, :, :]
        y_train = label_validation[train_loc, :]

        X_optical_vali = s2_validation[vali_loc, :, :, :]
        y_vali = label_validation[vali_loc, :]

        my_model.build_model("{}{}".format(MODEL_Prefix, _id))
        my_model.training((X_optical_train, y_train), (X_optical_vali, y_vali), 
                          model_name="{}{}".format(MODEL_Prefix, _id), epochs=40)

0 ; Train: [ 4824  4825  4826 ... 24116 24117 24118] | test: [   0    1    2 ... 4821 4822 4823]
Model Build Start!
Model Build Success!
start training!
Loss change at:  0 With AC:  0.03526666678488254 With Loss:  2.8539759286402866
Loss change at:  450 With AC:  0.376599999666214 With Loss:  2.4438574919060096
Loss change at:  500 With AC:  0.5121999967098236 With Loss:  1.685033739700444
Loss change at:  550 With AC:  0.7490000009536744 With Loss:  0.8122259720443296
Loss change at:  600 With AC:  0.8265999984741211 With Loss:  0.5579766341505161
Loss change at:  750 With AC:  0.8760000038146972 With Loss:  0.37411290693836624
Loss change at:  850 With AC:  0.8827333354949951 With Loss:  0.3506924311319987
Loss change at:  900 With AC:  0.9042666578292846 With Loss:  0.3052279534031503
Loss change at:  1400 With AC:  0.9011999940872193 With Loss:  0.2959449704409041
Loss change at:  1450 With AC:  0.9081333303451538 With Loss:  0.2761091444622818
Loss change at:  1750 With AC:  0.917

Loss change at:  500 With AC:  0.501852171421051 With Loss:  1.8564641178581147
Loss change at:  550 With AC:  0.6883478260040283 With Loss:  1.0038122465421173
Loss change at:  600 With AC:  0.8148434805870056 With Loss:  0.5813595995031994
Loss change at:  700 With AC:  0.8405826044082642 With Loss:  0.47878591682612837
Loss change at:  750 With AC:  0.8511999988555908 With Loss:  0.47089389209403254
Loss change at:  800 With AC:  0.8602434802055359 With Loss:  0.4330370450400476
Loss change at:  850 With AC:  0.8657826042175293 With Loss:  0.42198392056143436
Loss change at:  900 With AC:  0.8729217386245728 With Loss:  0.40553823467510114
Loss change at:  1000 With AC:  0.876921739578247 With Loss:  0.38183371812605693
Loss change at:  1050 With AC:  0.8848608684539795 With Loss:  0.37564663775303836
Loss change at:  1150 With AC:  0.8929217433929444 With Loss:  0.32812028926807446
Loss change at:  1550 With AC:  0.9037217378616333 With Loss:  0.28553022914177
Loss change at:  1700

# CV Test 

In [21]:
MODEL_Prefix = "SHUFFLE_TRUE_5CV"

In [22]:
CV_proba = np.zeros((test_num, CLASS_NUM))
for i in range(5):
    my_model.build_model("{}{}".format(MODEL_Prefix, i))
    CV_proba += my_model.batch_predict(s2_test, "{}{}".format(MODEL_Prefix, i))

Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_TRUE_5CV0.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_TRUE_5CV1.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_TRUE_5CV2.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_TRUE_5CV3.ckpt
Model Build Start!
Model Build Success!
INFO:tensorflow:Restoring parameters from ../ckpt/SHUFFLE_TRUE_5CV4.ckpt


In [23]:
CV_proba_max_loc = np.argmax(CV_proba, 1)
one_hot=tf.one_hot(CV_proba_max_loc,CLASS_NUM)
with tf.Session() as sess:
    _one_hot = sess.run(one_hot)
_one_hot = _one_hot.astype(np.int16)
pd.DataFrame(_one_hot).to_csv('../result/1229_deep_result_true{}.csv'.format(N), index=False, header=None)